In [ ]:
!pip install ns3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 8.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.6/206.6 kB 16.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached cppyy_backend-1.14.10-py2.py3-none-linux_x86_64.whl
  Using cached cppyy_cling-6.27.1-py2.py3-none-manylinux2014_x86_64.whl (32.4 MB)
  Created wheel for cppyy: filename=cppyy-2.4.2-py3-none-any.whl size=15352129 sha256=80e1f150e5885062a30ac708c599718ccc84d5304661c01a6cc0baf8a4731267
  Stored in directory: /root/.cache/pip/wheels/73/66/49/38da85078450bb1a5b63405aec0935de2e2991717394ac2f64
  Created wheel for CPyCppyy: filename=CPyCppyy-1.12.12-cp310-cp310-linux_x86_64.whl size=4182313 sha256=abd3e2d5b2e3

In [ ]:
from ns import ns


In [ ]:
# Create two nodes

# NodeContainer p2pNodes;
# p2pNodes.Create (2);
csmaNodes = ns.network.NodeContainer()
csmaNodes.Create(2)

# Create a CSMA helper to install the network interfaces and connect them via a physical link

# CsmaHelper csma;
# csma.SetChannelAttribute ("DataRate", StringValue ("100Mbps"));
# csma.SetChannelAttribute ("Delay", TimeValue (NanoSeconds (6560)));
csma = ns.csma.CsmaHelper()
csma.SetChannelAttribute("DataRate", ns.core.StringValue("100Mbps"))
csma.SetChannelAttribute("Delay", ns.core.TimeValue(ns.core.NanoSeconds(6560)))

# Install the network interfaces and connect them via a physical link

# NetDeviceContainer csmaDevices;
# csmaDevices = csma.Install (csmaNodes);
csmaDevices = csma.Install(csmaNodes)

# Instal the TCP/IP network stack

# InternetStackHelper stack;

  #  80   stack.Install (p2pNodes.Get (0));
  #  81   stack.Install (csmaNodes);
stack = ns.internet.InternetStackHelper()
stack.Install(csmaNodes)

# Create address then assign it to device
  #  83   Ipv4AddressHelper address;
  #  84   address.SetBase ("10.1.1.0", "255.255.255.0");
  #  85   Ipv4InterfaceContainer p2pInterfaces;
  #  86   p2pInterfaces = address.Assign (p2pDevices);
  #  87
  #  88   address.SetBase ("10.1.2.0", "255.255.255.0");
  #  89   Ipv4InterfaceContainer csmaInterfaces;
  #  90   csmaInterfaces = address.Assign (csmaDevices);

# Create an address pool for the network nodes
address = ns.internet.Ipv4AddressHelper()
address.SetBase(ns.network.Ipv4Address("10.1.2.0"), ns.network.Ipv4Mask("255.255.255.0"))
# Assign Ipv4 addresses to the network devices
csmaInterfaces = address.Assign(csmaDevices)

# Populate routing tables so that packages get properly routed
ns.internet.Ipv4GlobalRoutingHelper.PopulateRoutingTables()

In [ ]:
# Create, install and set the time the server application will run on the node 0
echoServer = ns.applications.UdpEchoServerHelper(9)
serverApps = echoServer.Install(csmaNodes.Get(0))
serverApps.Start(ns.core.Seconds(1.0))
serverApps.Stop(ns.core.Seconds(10.0))

# Create, configure, install and set the time the client application will run on the node 1
echoClient = ns.applications.UdpEchoClientHelper(csmaInterfaces.GetAddress(0).ConvertTo(), 9)
echoClient.SetAttribute("MaxPackets", ns.core.UintegerValue(10))
echoClient.SetAttribute("Interval", ns.core.TimeValue(ns.core.Seconds (1.0)))
echoClient.SetAttribute("PacketSize", ns.core.UintegerValue(1024))
clientApps = echoClient.Install(csmaNodes.Get(1))
clientApps.Start(ns.core.Seconds(2.0))
clientApps.Stop(ns.core.Seconds(10.0))

In [ ]:
# Cppyy bindings are not perfect and we need to create the callback via a Python-to-C++ wrapper
ns.cppyy.cppdef("""
    namespace ns3{
        const Callback<void,Ptr<const Packet>,const Address&,const Address&>
        make_sinktrace_callback(void(*func)(Ptr<const Packet>,const Address&,const Address&))
        {
            return MakeCallback(func);
        }
    }
""")

# We define our callback that will receive a packet smart-pointer and two addresses
def SinkTracer(packet: ns.Packet, src_address: ns.Address, dst_address: ns.Address) -> None:
    print(f"At {ns.Simulator.Now().GetSeconds():.0f}s, '{dst_address}' received packet"
          f" with {packet.__deref__().GetSerializedSize()} bytes from '{src_address}'")

# Create the callback calling the SinkTracer function defined above
sinkTraceCallback = ns.make_sinktrace_callback(SinkTracer)

# Register the callback to be called when the trace RxWithAddresses from UdpEchoServerHelper is executed
serverApps.Get(0).__deref__().TraceConnectWithoutContext("RxWithAddresses", sinkTraceCallback)

# Set the simulation duration to 11 seconds
ns.Simulator.Stop(ns.Seconds(11))

# Run the simulator
ns.Simulator.Run()
ns.Simulator.Destroy()

At 2s, '04-07-00:00:00:00:09:00:00' received packet with 60 bytes from '04-07-0a:01:02:02:01:c0:00'
At 3s, '04-07-00:00:00:00:09:00:00' received packet with 60 bytes from '04-07-0a:01:02:02:01:c0:00'
At 4s, '04-07-00:00:00:00:09:00:00' received packet with 60 bytes from '04-07-0a:01:02:02:01:c0:00'
At 5s, '04-07-00:00:00:00:09:00:00' received packet with 60 bytes from '04-07-0a:01:02:02:01:c0:00'
At 6s, '04-07-00:00:00:00:09:00:00' received packet with 60 bytes from '04-07-0a:01:02:02:01:c0:00'
At 7s, '04-07-00:00:00:00:09:00:00' received packet with 60 bytes from '04-07-0a:01:02:02:01:c0:00'
At 8s, '04-07-00:00:00:00:09:00:00' received packet with 60 bytes from '04-07-0a:01:02:02:01:c0:00'
At 9s, '04-07-00:00:00:00:09:00:00' received packet with 60 bytes from '04-07-0a:01:02:02:01:c0:00'
